<a href="https://colab.research.google.com/github/Yashmitha22/LLM/blob/main/model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install -U transformers accelerate


In [14]:
rm -rf ~/.cache/...

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/Phi-3-mini-4k-instruct"

# Load config
config = AutoConfig.from_pretrained(
    model_name,
    trust_remote_code=True
)

# ✅ FULL Phi-3 RoPE FIX (this is the missing part)
config.rope_scaling = {
    "type": "longrope",
    "short_factor": 1.0,
    "long_factor": 1.0
}

# Required by Phi-3
config.original_max_position_embeddings = config.max_position_embeddings

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [20]:
from transformers import pipeline
# Create a pipeline
generator = pipeline(
"text-generation",
model=model,
tokenizer=tokenizer,
return_full_text=False,
max_new_tokens=500,
do_sample=False
)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [28]:
messages = [
    {"role": "user", "content": "what is your name."}
]


In [29]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [30]:
output = generator(
    messages,
    use_cache=False
)

print(output[0]["generated_text"])

Both `max_new_tokens` (=500) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


KeyboardInterrupt: 